<a href="https://colab.research.google.com/github/fengivan/Fluids-Lab/blob/master/maintain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from time import sleep


import pandas
import time
import sys
import threading

def getDeltaP(seconds): #returns the average deltaP over the last "seconds"
    header = ["Time", "P1", "P2", "P3", "P4", "AvgP1", "AvgP2", "AvgFlow", "AvgTemp", "CMFlow", "CMDensity", "CM Temp", "Freq Set"]
    useless = ["Time", "P1", "P2",  "AvgP1", "AvgP2", "AvgFlow", "AvgTemp","CMFlow",  "CMDensity", "CM Temp", "Freq Set"]

    df=pandas.read_csv("/home/pi/Desktop/Shared/Fluids-Lab/data", names = header)
    df=(((df.tail(seconds*10+2)).head(seconds*10)).drop(useless,1)).astype(float)
    deltaP=df["P4"].mean()-df["P3"].mean()
    return deltaP


def addxanthan(amount):  #adds "amount grams of xcgum to the tank"
    addtime=round((amount/.8775)+.7)
    stopAdd = threading.Timer(addtime, stop)
    #print("Adding "+str(addtime*.82)+ " grams of XC Gum over "+ str(addtime) + " seconds", file=writeTo)
    outputs.write_single(0,745)
    stopAdd.start()
    
def stop():
    outputs.write_single(0,0)
    
def stable(seconds):   #determines if the deltaP is stable over the past x seconds, criteria is no more than 1% of deltaP measurements have more than a 10% difference with the average over the past x seconds.
    header = ["Time", "P1", "P2", "P3", "P4", "AvgP1", "AvgP2", "AvgFlow", "AvgTemp", "CMFlow", "CMDensity", "CM Temp", "Freq Set"]
    useless = ["Time", "P1", "P2",  "AvgP1", "AvgP2", "AvgFlow", "AvgTemp","CMFlow",  "CMDensity", "CM Temp", "Freq Set"]

    df=pandas.read_csv("editeddata", names = header)
    df=(((df.tail(seconds*10+2)).head(seconds*10)).drop(useless,1)).astype(float)
    avg=df["P4"].mean()-df["P3"].mean()
    p1=df["P4"].tolist()
    p2=df["P3"].tolist()
    count=0
    print(len(p1))
    for i in range(0,len(p1)):
      if (p1[i]-p2[i])>avg*1.1 or (p1[i]-p2[i])<avg*.9:
        count=count+1
      
    if(count<seconds/10):
      return True
    return False


def maintain(setpoint, savefile):
  lastaddtime=time.time()
  startTime=time.time()
  while True:
    curDeltaP=getDeltaP(10)
    print (str(time.time()-startTime)+","+str(getDeltaP(10)), file = savefile)
    if curDeltaP<setpoint and stable(300) and (time.time()-lastaddtime)>300:
      addxanthan((setpoint-curDeltaP)/.0146)
      lastaddtime=time.time()
    savefile.flush()
    sleep(10)



In [ ]:
stable(300)

3000


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


True